In [ ]:
#importations des bibliiotheques necessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from scipy.stats import zscore
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [ ]:
#chargement des donnees et affichage des 5 premieres lignes
df = pd.read_csv('5G_energy_consumption_dataset.csv')
df.head()

,Time,BS,Energy,load,ESMODE,TXpower
0,20230101 010000,B_0,64.275037,0.487936,0.0,7.101719
1,20230101 020000,B_0,55.904335,0.344468,0.0,7.101719
2,20230101 030000,B_0,57.698057,0.193766,0.0,7.101719
3,20230101 040000,B_0,55.156951,0.222383,0.0,7.101719
4,20230101 050000,B_0,56.053812,0.175436,0.0,7.101719


In [ ]:
#Affichages des infos sur les colonnes et statistiques et des valeurs nulles
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92629 entries, 0 to 92628
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Time     92629 non-null  object 
 1   BS       92629 non-null  object 
 2   Energy   92629 non-null  float64
 3   load     92629 non-null  float64
 4   ESMODE   92629 non-null  float64
 5   TXpower  92629 non-null  float64
dtypes: float64(4), object(2)
memory usage: 4.2+ MB


Time       0
BS         0
Energy     0
load       0
ESMODE     0
TXpower    0
dtype: int64

In [ ]:
#profiling
profile = ProfileReport(df, title="Rapport de Profiling")
profile.to_file("rapport_profiling.html")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00,  8.33it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#nettoyage des donnees
df = df.dropna()
df = df.drop_duplicates()
z_scores = np.abs(zscore(df.select_dtypes(include=np.number)))
df = df[(z_scores < 3).all(axis=1)]

In [ ]:
#supprimer les colonnes unitules
df.drop(columns=['Time'], inplace=True)
df

,BS,Energy,load,ESMODE,TXpower
0,B_0,64.275037,0.487936,0.0,7.101719
1,B_0,55.904335,0.344468,0.0,7.101719
2,B_0,57.698057,0.193766,0.0,7.101719
3,B_0,55.156951,0.222383,0.0,7.101719
4,B_0,56.053812,0.175436,0.0,7.101719
...,...,...,...,...,...
92624,B_1018,14.648729,0.087538,0.0,7.325859
92625,B_1018,14.648729,0.082635,0.0,7.325859
92626,B_1018,13.452915,0.055538,0.0,7.325859
92627,B_1018,13.602392,0.058077,0.0,7.325859


In [ ]:
#encoder les colonnes categorielles
df_encoded = pd.get_dummies(df,drop_first=True)
df_encoded

,Energy,load,ESMODE,TXpower,BS_B_1,BS_B_10,BS_B_100,BS_B_1006,BS_B_101,BS_B_1018,BS_B_102,BS_B_103,BS_B_104,BS_B_105,BS_B_106,BS_B_107,BS_B_108,BS_B_109,BS_B_11,BS_B_110,BS_B_111,BS_B_112,BS_B_113,BS_B_114,BS_B_115,BS_B_116,BS_B_117,BS_B_118,BS_B_12,BS_B_120,BS_B_121,BS_B_123,BS_B_124,BS_B_125,BS_B_126,BS_B_127,BS_B_128,BS_B_129,BS_B_130,BS_B_131,...,BS_B_923,BS_B_93,BS_B_930,BS_B_933,BS_B_934,BS_B_94,BS_B_945,BS_B_946,BS_B_947,BS_B_95,BS_B_951,BS_B_952,BS_B_953,BS_B_954,BS_B_96,BS_B_963,BS_B_966,BS_B_967,BS_B_968,BS_B_969,BS_B_97,BS_B_970,BS_B_971,BS_B_974,BS_B_98,BS_B_986,BS_B_987,BS_B_988,BS_B_989,BS_B_99,BS_B_990,BS_B_991,BS_B_992,BS_B_993,BS_B_994,BS_B_995,BS_B_996,BS_B_997,BS_B_998,BS_B_999
0,64.275037,0.487936,0.0,7.101719,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,55.904335,0.344468,0.0,7.101719,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,57.698057,0.193766,0.0,7.101719,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,55.156951,0.222383,0.0,7.101719,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,56.053812,0.175436,0.0,7.101719,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92624,14.648729,0.087538,0.0,7.325859,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [ ]:
# Définir X (features) et y (cible)
X = df_encoded.drop(columns=['Energy'])
y = df_encoded['Energy']  

# Séparation des données en données d'entraînement et données de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation (standardisation)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  

# Entraînement du modèle Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train_scaled, y_train)

# Prédiction sur les données de test
y_pred = ridge.predict(X_test_scaled)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Affichage des résultats
print('Model Ridge avec alpha = 0.1')
print('MSE:', mse)
print('RMSE:', rmse)
print('R2_SCORE:', r2)


Model Ridge avec alpha = 0.1
MSE: 7.973829665016539
RMSE: 2.823797029713102
R2_SCORE: 0.947533286237644


In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel("Valeurs réelles")
plt.ylabel("Valeurs prédites")
plt.title("Prédictions vs Réalité")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.show()

# Résidus
residuals = y_test - y_pred
sns.histplot(residuals, kde=True)
plt.title("Distribution des résidus")
plt.show()


In [ ]:
#  Interprétation des résultats :
#  MSE & RMSE faibles :
# Une erreur quadratique moyenne faible signifie que les prédictions du modèle sont proches des valeurs réelles.

# Le RMSE nous donne une idée de l’erreur moyenne en unités réelles de la variable cible (ici, Energy). Une RMSE de 2.82 signifie que les prédictions s’écartent en moyenne de ±2.82 unités d'énergie par rapport aux vraies valeurs, ce qui est généralement acceptable si les valeurs d'Energy sont dans une fourchette plus grande.

#  R² = 0.9475 :
# Le modèle explique environ 94,75 % de la variance des données.

# C’est un très bon score en régression.

# Plus ce score est proche de 1, plus le modèle est précis et fiable.

# Conclusion :
# Le modèle Ridge avec α=0.1 donne d'excellents résultats avec une faible erreur et un pouvoir explicatif élevé. Il semble donc bien adapté pour prédire la variable Energy à partir des caractéristiques fournies.
